<a href="https://colab.research.google.com/github/rootAkash/reinforcement_learning/blob/master/Soft%20AC/SAC_v2_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#much better version can be created where we dont use .predict but while generating labels 
#but we create the graph while generating labels for policy and value and qs and then the process becomes more efficient
#also instead of looping thru samples do vectorisation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from collections import deque
import random
from tqdm import tqdm
class Policy(nn.Module):

    def __init__(self,observation_size,action_size,hidden_units):
        super().__init__()
        self.observation_size=observation_size
        self.hidden_units=hidden_units
        self.action_size = action_size
        self.h1 = nn.Linear(self.observation_size, 2*self.hidden_units)  
        #self.h2 = nn.Linear(self.hidden_units,self.hidden_units)
        #self.h3 = nn.Linear(self.hidden_units,self.hidden_units)
        #self.h4 = nn.Linear(self.hidden_units,self.hidden_units)
        #self.h5 = nn.Linear(self.hidden_units,self.hidden_units)
        self.h6 = nn.Linear(2*self.hidden_units,self.hidden_units)
        self.mu = nn.Linear(self.hidden_units,self.action_size)
        self.sigma = nn.Linear(self.hidden_units,self.action_size)
    def forward(self, x):
        x = F.relu(self.h1(x))
        #x = F.relu(self.h2(x))
        #x = F.relu(self.h3(x))
        #x = F.relu(self.h4(x))
        #x = F.relu(self.h5(x))
        x = F.relu(self.h6(x))
        mus = torch.tanh(self.mu(x))
        sigs= F.softplus(self.sigma(x))
        sigs= torch.clamp(sigs, min=0.001, max=100)#1e-22 , 1e+02
        return mus , sigs
    def predict(self, x):
        with torch.no_grad():
          output=self.forward(x)
        return output  

class Value(nn.Module):
    def __init__(self,observation_size,hidden_units):
        super().__init__()
        self.observation_size=observation_size
        self.hidden_units=hidden_units
        self.h1 = nn.Linear(self.observation_size, 2*self.hidden_units)  
        #self.h2 = nn.Linear(self.hidden_units,self.hidden_units)
        #self.h3 = nn.Linear(self.hidden_units,self.hidden_units)
        #self.h4 = nn.Linear(self.hidden_units,self.hidden_units)
        #self.h5 = nn.Linear(self.hidden_units,self.hidden_units)
        self.h6 = nn.Linear(2*self.hidden_units,self.hidden_units)
        self.v = nn.Linear(self.hidden_units,1)
    def forward(self, x):
        x = F.relu(self.h1(x))
        #x = F.relu(self.h2(x))
        #x = F.relu(self.h3(x))
        #x = F.relu(self.h4(x))
        #x = F.relu(self.h5(x))
        x = F.relu(self.h6(x))
        vout = self.v(x)
        return vout
    def predict(self, x):
        with torch.no_grad():
          output=self.forward(x)
        return output    
class Q_net(nn.Module):

    def __init__(self,observation_size,action_size,hidden_units):
        super().__init__()
        self.observation_size=observation_size
        self.hidden_units=hidden_units
        self.action_size = action_size
        self.h1 = nn.Linear(self.observation_size, self.hidden_units) 
        self.a1 = nn.Linear(self.action_size, self.hidden_units) 
        self.h2 = nn.Linear(self.hidden_units*2,self.hidden_units)
        #self.h3 = nn.Linear(self.hidden_units,self.hidden_units)
        #self.h4 = nn.Linear(self.hidden_units,self.hidden_units)
        #self.h5 = nn.Linear(self.hidden_units,self.hidden_units)
        #self.h6 = nn.Linear(self.hidden_units,self.hidden_units)
        self.q = nn.Linear(self.hidden_units,1)
    def forward(self, x,a):
        x = F.relu(self.h1(x))
        a = F.relu(self.a1(a))
        x = torch.cat([x,a], dim=1)
        x = F.relu(self.h2(x))
        #x = F.relu(self.h3(x))
        #x = F.relu(self.h4(x))
        #x = F.relu(self.h5(x))
        #x = F.relu(self.h6(x))
        qout = self.q(x)
        return qout
    def predict(self, x,a):
        with torch.no_grad():
          output=self.forward(x,a)
        return output



In [2]:
def remember(s,a,r,ns,d):
  #s=s.ravel()
  #ns=ns.ravel()
  memory.append([s,a,r,ns,d])
def sample_games(buffer,batch_size):
  # Sample game from buffer either uniformly or according to some priority
  #print("samplig from .",len(buffer))
  return list(np.random.choice(len(buffer),batch_size))
def soft_update(target, source, tau):
  for target_param, param in zip(target.parameters(), source.parameters()):
    target_param.data.copy_(target_param.data * (1.0 - tau) + param.data * tau)
  return target
def get_log_pdf(x, mean, std):
  sigma_sq = (std)**2
  pdf= 1./(2*3.1415926*sigma_sq)**0.5 * math.exp(-(float(x)-float(mean))**2/(2.*sigma_sq))
  log_pdf = np.log(pdf) 
  return log_pdf
def get_entropy(x, mean, std,Act):
  sigma_sq = (std)**2 #variace
  pdf= 1./(2*3.1415926*sigma_sq)**0.5 * math.exp(-(float(x)-float(mean))**2/(2.*sigma_sq))
  ent = np.log(pdf) - np.log((1- (Act)**2 )+ 1e-07) # pdf of squashed guassian Act is after squashing  by tanh and x is before that
  #log a - log b = log a/b
  return -ent
def get_log_pdf_multi(x,mean,std):
  k= x.shape[0] #actio dim
  pi = 3.1415926
  cov = std**2
  det = np.prod(cov)
  norm_const = 1.0/ ( np.power(2*pi,k/2) * np.power(det,0.5) )
  prod  = (1/cov)*np.square(x - mean)
  prod2 =np.sum(prod) 
  pdf = norm_const * np.exp( -0.5 *prod2)
  return np.log(pdf+1e-07)

def get_entropy_multi(x, mean, std,Act):
  #log pdf (squashed guassian) = log pdfguassian(mupolicy,sigma_policy) - sum of  log(1-A**2) ; where each A is component of tanh squahed action vector 
  log_pdf_final = get_log_pdf_multi(x,mean,std) - np.sum(np.log(1- (Act)**2 +1e-07))
  return -log_pdf_final



def replay_and_train(policy,value,t_value,q_1,q_2,popt,vopt,qopt,size=128):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  gamma=0.95
  mse = nn.MSELoss()
  alpha = 1.0
  sample_size=size
  if len(memory) < sample_size:
    return
  samples=random.sample(memory,sample_size)
  y_q=[]
  states =[]
  y_v=[]
  Es=[]
  Acts = []
  for sample in samples:
    s,a,r,ns,d=sample
    yq = r + gamma*(1-d)*t_value.predict(torch.tensor([ns]).float().to(device)).cpu().numpy()[0]
    new_mu,new_sig = policy.predict(torch.tensor([s]).float().to(device))
    new_mu,new_sig = new_mu.cpu().numpy(),new_sig.cpu().numpy()
    #print(new_mu,new_sig,new_mu[0],new_sig[0])
    E =np.random.multivariate_normal(np.zeros_like(new_mu[0]),np.diag(np.ones_like(new_mu[0])))#np.random.normal(mu=0,std_dev=1) diag cov of 1 is same as std dev of 1
    #print(E)
    Act_guassian = new_mu[0] + new_sig[0]*E # guassian action using reparametrisation
    new_Action  =np.tanh(Act_guassian)  #final squashed guassian action
    entropy = get_entropy_multi(Act_guassian,new_mu,new_sig,new_Action)
    #print(s,new_Action,s.shape,new_Action.shape,torch.tensor([s]).float(),torch.tensor([s]).float().shape)
    yv = min(q_1.predict(torch.tensor([s]).float().to(device),torch.tensor([new_Action]).float().to(device)).cpu().numpy()[0],
             q_2.predict(torch.tensor([s]).float().to(device),torch.tensor([new_Action]).float().to(device)).cpu().numpy()[0]) + alpha*entropy
    #print(yv,yq)
    Es.append(E)#for policy loss and reparameterisation
    y_q.append(yq)
    y_v.append(yv)
    states.append(s)
    Acts.append(a)
  Es = torch.tensor(Es).float().to(device)
  y_q = torch.tensor(y_q).float().to(device)
  y_v = torch.tensor(y_v).float().to(device)
  states = torch.tensor(states).float().to(device)
  Acts = torch.tensor(Acts).float().to(device)
  #training nets 
  #train q nets
  qloss = mse(q_1(states,Acts),y_q) + mse(q_2(states,Acts),y_q)
  qopt.zero_grad()                                                                                                          #    
  qloss.backward()                                                                                                         #
  qopt.step() 
  #train v net
  vloss = mse(value(states),y_v)
  vopt.zero_grad()                                                                                                          #    
  vloss.backward()                                                                                                         #
  vopt.step() 
  #train policy 
  #we dont have to do this additional computation actually since its done before but that was in loop
  #when vectorised no need to do this 
  mus,sigs = policy(states)
  #print(mus.shape,Es.shape,sigs.shape,(Es*sigs).shape)
  x= mus + Es*sigs
  actions = torch.tanh(x)
  k= x.shape[1] #action dim
  pi = 3.1415926
  cov = sigs**2
  det = torch.prod(cov,dim=1,keepdim=True)
  norm_const = 1.0/ ( np.power(2*pi,k/2) * torch.pow(det,0.5) )
  prod  = (1/cov)*torch.square(x - mus)
  prod2 =torch.sum(prod,dim=1,keepdim=True) 
  pdf = norm_const * torch.exp( -0.5 *prod2)
  final_log_pdf = torch.log(pdf+1e-07)  - torch.sum(torch.log(1- torch.square(actions) +1e-07),dim=1,keepdim=True)
  entropy = - final_log_pdf
  policy_objective = q_1(states,actions)#state from buffer action from recent policy and not from buffer to train to maximise q
  final_policy_objective = policy_objective + alpha*entropy # maximise this
  final_policy_loss = - torch.mean(final_policy_objective)  #therefore minimise this
  popt.zero_grad()                                                                                                          #    
  final_policy_loss.backward()                                                                                                         #
  popt.step() 
  
  #train t_value
  soft_update(target=t_value, source=value, tau=0.01)

  


    

In [ ]:
import gym

memory=deque(maxlen=5000000)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#env,networks ad optimisers
#env=gym.make('Pendulum-v0')
env=gym.make('MountainCarContinuous-v0')

env=env.unwrapped

s_dim = env.observation_space.shape[0]
print(s_dim)
a_dim = env.action_space.shape[0]
print(a_dim)
a_bound = env.action_space.high[0]
print(a_bound)

policy = Policy(s_dim,a_dim,100).to(device)
value = Value(s_dim,100).to(device)
t_value = Value(s_dim,100).to(device)
soft_update(target=t_value, source=value, tau=1)
q_1 = Q_net(s_dim,a_dim,100).to(device)
q_2 = Q_net(s_dim,a_dim,100).to(device)
qopt = optim.Adam(list(q_1.parameters()) + list(q_2.parameters()),lr=0.001,betas=(0.5, 0.999))
popt = optim.Adam(policy.parameters(),lr=0.001,betas=(0.5, 0.999))
vopt = optim.Adam(value.parameters(),lr=0.001,betas=(0.5, 0.999))        
max_steps=5000
episodes = 5000
steps = 3500 
ctr = 0
render =False
train_iter = 1000  # 1000, batch of 8/16 works best for now

for ep in range(episodes):
	s = env.reset()
	done=False
	rews=0
	if ep>1500:
		render=1
	for step in range(steps):
		if done:
			s = env.reset()
		if render:
			env.render()	
   
		new_mu,new_sig = policy.predict(torch.tensor([s]).float().to(device))
		
		mu,sig=new_mu.cpu().numpy(),new_sig.cpu().numpy()
		E =np.random.multivariate_normal(np.zeros_like(mu[0]),np.diag(np.ones_like(sig[0])))#np.random.normal(0,1)

		Action  =np.tanh(mu[0] + sig[0]*E)
		if ep < 10 :
			# for some additional exploration not necesserily needed
			if E > 0.5:
				Action = np.clip(a_bound*E,-a_bound,a_bound)		
		s_,r,done,_=env.step(Action*a_bound)
		if done :
			r =r+10000	# to encourage reaching target more
			print("reached")
		remember(s,Action,r,s_,done)
		rews+=r	
		ctr+=1
		s=s_
	print("episode: "+str(ep)+ " rews: "+str(rews))		
	print("training")
	for i in  range(train_iter):
		replay_and_train(policy,value,t_value,q_1,q_2,popt,vopt,qopt,size=16)
		if i % (train_iter//10)==0:
			print('.',end='')
	print('|')	

2
1
1.0
episode: 0 rews: -125.45088658864003
training
..........|
episode: 1 rews: -142.58479914934682
training
..........|
episode: 2 rews: -148.28171052453268
training
..........|
episode: 3 rews: -139.23786982325126
training
..........|
episode: 4 rews: -148.41020043911286
training
..........|
reached
episode: 5 rews: 9952.47542697799
training
..........|
episode: 6 rews: -153.64919026695753
training
..........|
reached
episode: 7 rews: 9952.220636184984
training
..........|
episode: 8 rews: -152.1769298895317
training
..........|
reached
episode: 9 rews: 9957.274953742517
training
..........|
episode: 10 rews: -116.83564227277432
training
..........|
episode: 11 rews: -121.55723536837017
training
..........|
episode: 12 rews: -115.01780376123126
training
..........|
episode: 13 rews: -118.45500642149209
training
..........|
episode: 14 rews: -46.191101952046225
training
..........|
episode: 15 rews: -129.63319782408317
training
..........|
episode: 16 rews: -120.70486565544185
trai

In [ ]:
import gym

memory=deque(maxlen=5000000)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#env,networks ad optimisers
#env=gym.make('Pendulum-v0')
env=gym.make('MountainCarContinuous-v0')

env=env.unwrapped

s_dim = env.observation_space.shape[0]
print(s_dim)
a_dim = env.action_space.shape[0]
print(a_dim)
a_bound = env.action_space.high[0]
print(a_bound)

policy = Policy(s_dim,a_dim,100).to(device)
value = Value(s_dim,100).to(device)
t_value = Value(s_dim,100).to(device)
soft_update(target=t_value, source=value, tau=1)
q_1 = Q_net(s_dim,a_dim,100).to(device)
q_2 = Q_net(s_dim,a_dim,100).to(device)
qopt = optim.Adam(list(q_1.parameters()) + list(q_2.parameters()),lr=0.001,betas=(0.5, 0.999))
popt = optim.Adam(policy.parameters(),lr=0.001,betas=(0.5, 0.999))
vopt = optim.Adam(value.parameters(),lr=0.001,betas=(0.5, 0.999))        
max_steps=5000
episodes = 500
train_iter = 1000  # 1000, batch of 8/16 works best for now
for e in range(episodes):
  done = False
  s = env.reset()
  rew = 0 
  stp=0
  while not done:
    new_mu,new_sig = policy.predict(torch.tensor([s]).float().to(device))
    new_mu,new_sig=new_mu.cpu().numpy(),new_sig.cpu().numpy()
    #print(new_mu,new_sig,new_mu[0],new_sig[0])
    E =np.random.multivariate_normal(np.zeros_like(new_mu[0]),np.diag(np.ones_like(new_mu[0])))#np.random.normal(mu=0,std_dev=1) diag cov of 1 is same as std dev of 1
    Act_guassian = new_mu[0] + new_sig[0]*E # guassian action using reparametrisation
    act  =np.tanh(Act_guassian)
    #if e<10:
    #  if np.random.normal(0,1)>0.5:
    #    act=E	#for agressive exploration in mountain car
    #  print(E) 
    s_,r,done,_=env.step(act*a_bound)
    #r = s_[0]*10 # for mountain car reward hacking 0 pos ,1 vel
    if done:
      print("reached")
      #r=10000 # for mountain car
    if stp>max_steps:
      done = True  
    remember(s,act,r,s_,done)
    s=s_
    rew+=r
    stp+=1
  print(e,rew)
  if e>20:
    print("training")
    for i in  tqdm(range(train_iter)):
      replay_and_train(policy,value,t_value,q_1,q_2,popt,vopt,qopt,size=8)
